In [2]:
from custom_rag_loader import DbSupportedEmbeddingModels, RagConfig, SupportedModels, load_llm_rag_model

llm, db = load_llm_rag_model(
    RagConfig(
        model=SupportedModels.Llama3, 
        n_ctx=4096,
        db_embedding_model=DbSupportedEmbeddingModels.Paraphrase_MiniLM_L6_v2
        )
    )


/home/tpllmws23/environments/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tpllmws23/environments/rag/lib/python3.11/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! chat_format is not default parameter.
                chat_format was transferred to model_kwargs.
                Please confirm that chat_format is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /home/tpllmws23/llms/Meta-Llama-3-8B-Instruct.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general

In [8]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores.chroma import Chroma

def load_db():
    config = RagConfig(
        model=SupportedModels.Mistral,
        db_embedding_model=DbSupportedEmbeddingModels.Paraphrase_MiniLM_L6_v2
    )
    
    embedding = SentenceTransformerEmbeddings(model_name=config.db_embedding_model.value.split("/")[-1])
    db = Chroma(
        persist_directory=config.db_path, 
        collection_name=config.db_collection, 
        embedding_function=embedding
    )

    return db

In [12]:
db = load_db()

prompt = "Welche Dokumente können als Nachweis für deutsche Sprachkenntnisse akzeptiert werden?"
documents = db.similarity_search(prompt, k=5)
i = 42

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.globals import set_debug
from operator import itemgetter
from llama_cpp import Llama

# set_debug(True)

template = """Answer the following question based only on the provided context. Always return the source of an information. Return the answer in Markdown format:
{context}

Question: {question}"""

prompt = ChatPromptTemplate.from_template(template)

# retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})
# retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

def format_docs(docs):
    context =""
    for doc in docs:
        context += "Content: \n" + doc.page_content + "\n"
        context += "Source: \n" + str(doc.metadata['file_path']) + "\n\n\n"
    return context

def simple_rag_chain(question: str):
    docs = db.search(question, 'similarity', k=5)
    # docs = retriever.get_relevant_documents(question)
    formatted_docs = format_docs(docs)

    final_prompt = prompt.format(context= formatted_docs, question= question)
    llm.invoke(final_prompt)

In [6]:
from IPython.display import display, Markdown

response = simple_rag_chain("Welche Dokumente können als Nachweis für deutsche Sprachkenntnisse akzeptiert werden?")
display(Markdown(response))



Answer: Die folgenden

Llama.generate: prefix-match hit


 Dokumente können als Nachweis für deutsche Sprachkenntnisse akzeptiert werden:

1. "Telc Deutsch C1 Hochschule" oder eine äquivalente Sprachprüfung gemäß der Rahmenordnung über Deutsche Sprachprüfungen für das Studium an deutschen Hochschulen (RO-DT).
2. Sprachnachweise für den gewählten Masterstudiengang, die durch die Bewerberin oder den Bewerber bis zum Bewerbungsschluss nicht vorgelegt werden können, können bis zum Vorlesungsbeginn des Semesters gemäß Terminplan der Hochschule Konstanz, für das der Antrag auf Zulassung gestellt wurde, nachgereicht werden.

3. Für Zeitstudierende gelten die Regelungen in § 10 Zulassungs- und Immatrikulationsordnung (ZIO) der Hochschule Konstanz.


llama_print_timings:        load time =     957.62 ms
llama_print_timings:      sample time =      94.36 ms /   228 runs   (    0.41 ms per token,  2416.23 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    3117.64 ms /   228 runs   (   13.67 ms per token,    73.13 tokens per second)
llama_print_timings:       total time =    4074.73 ms /   229 tokens


<IPython.core.display.Markdown object>

In [9]:
query = "Answer the following question based only on the provided context with citation. Return the answer in Markdown format:\nContent: \nSemesters bekannt zu geben. Die Masterarbeit ka nn in englischer Sprache verfasst werden.\nSource: \n/home/tpllmws23/llms/main_data/Wahlpflichtmodule_Master-Informatik_WS23_Stand_21-Sep-2023.pdf\n\n\nContent: \nHochschule Konstanz Modulhandbuch des Studiengangs\nFakultät InformatikPlaceholder Informatik, Master of Science\nModul User Experience Engineering\nModul-Koordination Start Modul-Kürzel/-Nr. ECTS-Punkte Arbeitsaufwand\nProf. Dr. A. Heß SS UEE 5 150 h\n Dauer SWS Kontaktzeit Selbststudium\n 1 Semester 4 45 h 105 h\nEinsatz des Moduls im\nStudiengangAngestrebter\nAbschlussModul-Typ\n(PM/WPM)Beginn im\nStudiensemesterSPO-Version /\nJahr\nMSI M.Sc. WPM A/B SPO 5 / 2020\nInhaltliche Teilnahme\nVoraussetzung\nVerwendbarkeit des Moduls\nim o.g. StudiengangAls Vorkenntnis erforderlich für Modul:\nSinnvoll zu kombinieren mit Modul:\nPüfungsleistungen\ndes ModulsBenotete PrüfungUnbenotete\nPrüfungUnbenoteter\nLeistungsnachweis\nModulprüfung (MP) K90 \nModulteilprüfung (MTP) \nZusammensetzung\nder Endnote S Note der benoteten Modul(teil)prüfung\n £ ECTS-gewichtetes, arithmetisches Mittel der benoteten Modulteilprüfungen\n £ Sonstiges:\nLernziele des\nModuls\nFachliche Kompetenzen\nEine positive User Experience gilt als wichtiges Qualitätskriterium für den Erfolg interaktiver\nSoftwareprodukte.  Doch was verbirgt sich hinter dem Begriff “User Experience” und wie lässt sich die\nUser Experience konstruktiv in einem Softwareentwicklungsprojekt sicherstellen bzw. überprüfen?\nSource: \n/home/tpllmws23/llms/main_data/Wahlpflichtmodule_Master-Informatik_WS23_Stand_21-Sep-2023.pdf\n\n\nContent: \nHochschule Konstanz  \nTechnik, Wirtschaft und Gestaltung  \n \n \nSeite 10 von 43 (3) 1Bewerberinnen und Bewerber nach Absatz 1 werden nach ihrer Eignung für den \ngewählten Studiengang und sich typischerweise anschließende Berufstätigkeiten gemäß § 9 \nim Auswahlverfahren  ausgewählt.\nSource: \n/home/tpllmws23/llms/main_data/Wahlpflichtmodule_Master-Informatik_WS23_Stand_21-Sep-2023.pdf\n\n\nContent: \nHochschule Konstanz Modulhandbuch des Studiengangs\nFakultät InformatikPlaceholder Informatik, Master of Science\nModul ITM03 ERP-Geschäftsprozesse\nModul-Koordination Start Modul-Kürzel/-Nr. ECTS-Punkte Arbeitsaufwand\nProf. Dr. R. Martin SS ERPG/ITM03 5 150 h\n Dauer SWS Kontaktzeit Selbststudium\n 1 Semester 3 45 h 105 h\nEinsatz des Moduls im\nStudiengangAngestrebter\nAbschlussModul-Typ\n(PM/WPM)Beginn im\nStudiensemesterSPO-Version /\nJahr\nMSI M.Sc. WPM A/B SPO 5 / 2020\nInhaltliche Teilnahme\nVoraussetzung\nVerwendbarkeit des Moduls\nim o.g. StudiengangAls Vorkenntnis erforderlich für Modul:\nSinnvoll zu kombinieren mit Modul: ERPS/ITM04\nPüfungsleistungen\ndes ModulsBenotete PrüfungUnbenotete\nPrüfungUnbenoteter\nLeistungsnachweis\nModulprüfung (MP) \nModulteilprüfung (MTP) SP (LP, PR, AB) \nZusammensetzung\nder Endnote S Note der benoteten Modul(teil)prüfung\n £ ECTS-gewichtetes, arithmetisches Mittel der benoteten Modulteilprüfungen\n £ Sonstiges:\nLernziele des\nModuls\nDie Studierenden verstehen, wie grundlegende betriebliche Geschäftsprozesse unterschiedlicher\nUnternehmenstypen in ERP-Systemen abgebildet werden.\nSource: \n/home/tpllmws23/llms/main_data/Wahlpflichtmodule_Master-Informatik_WS23_Stand_21-Sep-2023.pdf\n\n\nContent: \nFür die Auswahlentscheidung wird unter den Bewerbern und Bewerberinnen eine Rangliste \nnach einer Auswahlnote erstellt, die wie folgt ermittelt wird. Von der Teilnote 1 wird die Teilnote \n2 nach Abs. 2 Nr. 4 abgezogen.  \n(5) Ausländerquote gemäß Anlage 8 HZVO  \nNicht zutreffend.\nSource: \n/home/tpllmws23/llms/main_data/Wahlpflichtmodule_Master-Informatik_WS23_Stand_21-Sep-2023.pdf\n\n\nContent: \n(3) 1Bei der Anerkennung von akademischen Graden, die außerhalb der Bundesrepublik \nDeutschland erworben wurden, sind die von der Kultusministerkonferenz und der \nHochschulrektorenkonferenz gebilligten Äquivalenzvereinbarungen sowie die Vereinbarungen  \nim Rahmen von Hochschulpartnerschaften zu beachten.\nSource: \n/home/tpllmws23/llms/main_data/Wahlpflichtmodule_Master-Informatik_WS23_Stand_21-Sep-2023.pdf\n\n\nContent: \nAuswahlgespräch mindestens mit der Note „4,0“ abgeschlos sen haben, eine Rangliste nach \nder Auswahlnote gemäß Abs. 2 Nr. 1 erstellt.  \n(5) Ausländerquote gemäß Anlage 8 HZVO \nNicht zutreffend.\nSource: \n/home/tpllmws23/llms/main_data/Wahlpflichtmodule_Master-Informatik_WS23_Stand_21-Sep-2023.pdf\n\n\nContent: \n- eine Bescheinigung übe r den Aufenthalt im englischsprachigen Ausland, der einen \nZeitraum von mindestens sechs Monaten bzw. einem Studiensemester umfasst.  \nDie Vorlage anderer geeigneter Nachweise ist möglich.  \n(2) Auswahlkriterien nach § 9 Abs. 2  \n1. Ergebnis eines Auswahlgesprächs  \nNicht zutreffend.  \n2. Leistungen, die mit der Abschlussprüfung des grundständigen Studiums nach Abs. 1 \ni. V. m. § 5 Abs. 1 Nr. 1 nachgewiesen sind \nDie Durchschnittsnote der Abschlussprüfung des grundständigen Hochschulstudiums nach \nAbs. 1 bildet die Auswahlnot e für die Auswahlentscheidung. Abweichend von Satz 1 bildet in \nden Fällen des § 3 Abs. 2 Nr. 1 Satz 2 die Durchschnittsnote nach § 3 Abs. 2 Nr. 1 Satz 3 die \nTeilnote 1. Bei ausländischen Bildungsnachweisen ist die Durchschnittsnote nach deutscher \nDeutung als Teilnote 1 zu berücksichtigen. Zusätzlich werden die Einzelnoten folgender \nFächer der Abschlussprüfung des grundständigen Hochschulstudiums, die über die Eignung \nfür den gewählten Studiengang besonderen Aufschluss geben, für die Auswahl herangezogen:  \n- Mathematik,  \n- Projektmanagement,  \n- Qualitätsmanagement,  \n- Allgemeine Betriebswirtschaftslehre / Kostenrechnung.\nSource: \n/home/tpllmws23/llms/main_data/Wahlpflichtmodule_Master-Informatik_WS23_Stand_21-Sep-2023.pdf\n\n\nContent: \n- Mathematik,  \n- Projektmanagement,  \n- Qualitätsmanagement,  \n- Allgemeine Betriebswirtschaftslehre / Kostenrechnung.  \nDabei wird eine Note zwischen 1,0 und 1,7 in einem der o. g. Fächer jeweils mit dem Wert 0,1 bewertet. Die kumulierte Gesamtzahl bildet die Teilnote 2.  \n3. Ergebnis eines fachspezifischen Studienfähigkeitstests  \nNicht zutreffend.\nSource: \n/home/tpllmws23/llms/main_data/Wahlpflichtmodule_Master-Informatik_WS23_Stand_21-Sep-2023.pdf\n\n\nContent: \n Technische Mechanik (Dynamik),  \n Thermodynamik,  \n Messtechnik,  \n Regelungstechnik,  \n Fahrzeugtechnik.  \nDabei wird eine Note zwischen 1,0 und 1,7 in einem der o. g. Fächer jeweils mit dem Wert 0,1 \nbewertet. Die kumulierte Gesamtzahl bildet die Teilnote 2. \n3. Ergebnis eines fachspezifischen Studienfähigkeitstests  \nNicht zutreffend.\nSource: \n/home/tpllmws23/llms/main_data/Wahlpflichtmodule_Master-Informatik_WS23_Stand_21-Sep-2023.pdf\n\n\n\n\nQuestion: Was sind die zulassungsvorraussetzungen um an der HTWG MSI mit der Vertiefungsrichtung Autonome Systeme zu studieren?"
print(query)

Answer the following question based only on the provided context with citation. Return the answer in Markdown format:
Content: 
Semesters bekannt zu geben. Die Masterarbeit ka nn in englischer Sprache verfasst werden.
Source: 
/home/tpllmws23/llms/main_data/Wahlpflichtmodule_Master-Informatik_WS23_Stand_21-Sep-2023.pdf


Content: 
Hochschule Konstanz Modulhandbuch des Studiengangs
Fakultät InformatikPlaceholder Informatik, Master of Science
Modul User Experience Engineering
Modul-Koordination Start Modul-Kürzel/-Nr. ECTS-Punkte Arbeitsaufwand
Prof. Dr. A. Heß SS UEE 5 150 h
 Dauer SWS Kontaktzeit Selbststudium
 1 Semester 4 45 h 105 h
Einsatz des Moduls im
StudiengangAngestrebter
AbschlussModul-Typ
(PM/WPM)Beginn im
StudiensemesterSPO-Version /
Jahr
MSI M.Sc. WPM A/B SPO 5 / 2020
Inhaltliche Teilnahme
Voraussetzung
Verwendbarkeit des Moduls
im o.g. StudiengangAls Vorkenntnis erforderlich für Modul:
Sinnvoll zu kombinieren mit Modul:
Püfungsleistungen
des ModulsBenotete PrüfungUnbenotet

In [5]:

retriever.get_relevant_documents("Bewerbung MSI Autonome Systeme")

[Document(page_content='- Wahlpflichtmodul Autonome Systeme 2  X  (3) 3  \n        \n9 Wahlpflichtmodul Autonome Systeme 3 2) WPM   3    \n - Wahlpflichtmodul Autonome Systeme 3  X  (3) 3  \n        \n10 Wahlpflichtmodul Autonome Systeme 4 2) WPM   3    \n - Wahlpflichtmodul Autonome Systeme 4  X  3 (3)  \n        \n11 Wahlpflichtmodul Autonome Systeme 5 2) WPM   3    \n - Wahlpflichtmodul Autonome Systeme 5  X  3 (3)  \n        \n12 Wahlpflichtmodul 3) WPM   6    \n - Wahl pflichtmodul  X  3 3  \n        \n Masterarbeit       \n        \n Mündliche Masterprüfun g       \n        \n Summe gesamtes Studium    33    \n \n1)  Entsprechend des Angebots der Fakultät sind die in Klammern angegebenen Veranstaltungen entweder in \nSemester A oder B zu absolvieren. \n2)  Lehrveranstaltungen sind aus dem W ahlpflichtangebot der Vertiefu ngsrichtung Autonome Systeme im \nStudiengang MSI zu wählen.  \n3)  Lehrveranstaltungen sind aus dem Wahlpflichtangeb ot aller drei Vertiefungsrichtungen und dem